In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## Process the text
### Vectorize the text

In [6]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [10]:
len(ids_from_chars.get_vocabulary())

66

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Create training examples and targets

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target :", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target : b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches

In [24]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [25]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
    )

### Try the model

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 66)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [31]:
sampled_indices

array([60, 43, 56, 61, 23,  8, 38, 49, 31, 55, 11, 51, 31, 22, 31, 35, 21,
       28,  7,  4, 64, 15,  8, 10, 48, 42,  3, 55, 13, 39, 60,  4,  7, 59,
       38, 54, 59, 15, 11, 42, 33, 38, 13, 21, 34, 50, 19, 40, 47, 48, 15,
       62, 41, 49, 15, 31, 58, 48, 62, 64, 35, 63,  7,  8, 34, 23, 47, 23,
        3, 36, 16, 19, 32, 21, 51, 59, 54, 15,  2,  6, 59, 12, 21, 26, 33,
       29, 36, 37, 63, 41, 12, 25, 62, 33, 52, 18, 28, 62, 45, 34])

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'mberland.\n\nNORTHUMBERLAND:\nThe commons will not then be satisfied.\n\nKING RICHARD II:\nThey shall be s'

Next Char Predictions:
 b"udqvJ-YjRp:lRIRVHO,$yB-3ic!p?Zu$,tYotB:cTY?HUkFahiBwbjBRsiwyVx,-UJhJ!WCFSHltoB 't;HMTPWXxb;LwTmEOwfU"


## Train the model

In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189123, shape=(), dtype=float32)


In [35]:
tf.exp(example_batch_mean_loss).numpy()

65.96493

In [36]:
model.compile(
    optimizer="adam",
    loss=loss
)

### Configure checkpoints

In [37]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

## Execute the training

In [54]:
EPOCHS = 20

In [55]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 10s 50ms/step - loss: 0.4689
Epoch 2/20
172/172 [==============================] - 11s 50ms/step - loss: 0.4625
Epoch 3/20
172/172 [==============================] - 11s 51ms/step - loss: 0.4567
Epoch 4/20
172/172 [==============================] - 11s 53ms/step - loss: 0.4521
Epoch 5/20
172/172 [==============================] - 12s 54ms/step - loss: 0.4448
Epoch 6/20
172/172 [==============================] - 11s 53ms/step - loss: 0.4438
Epoch 7/20
172/172 [==============================] - 11s 52ms/step - loss: 0.4426
Epoch 8/20
172/172 [==============================] - 10s 52ms/step - loss: 0.4347
Epoch 9/20
172/172 [==============================] - 10s 52ms/step - loss: 0.4320
Epoch 10/20
172/172 [==============================] - 13s 55ms/step - loss: 0.4274
Epoch 11/20
172/172 [==============================] - 12s 54ms/step - loss: 0.4273
Epoch 12/20
172/172 [==============================] - 13s 54ms/step - loss: 0.4233
E

## Generate text

In [62]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index
        values = [-float('inf')] * len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape = [len(ids_from_chars.get_vocabulary())]
    )
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
    # Only use the last prediction
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state
    return predicted_chars, states

In [63]:
one_step_model = OneStep(
    model,
    chars_from_ids,
    ids_from_chars
)

In [65]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The early more deny:
How have you news of our here prince, be gone!
But yet my land I swear thy name, but that I do.

MERCUTIO:
Any, to appear, and now 'tis this day say: what
A joy it like your good friendships may beseem him come.

ONIO:
Haply come; fellow Madam:
That is no bitter'd traitors? but the strong
Their means to-morrow mocks, or royal father.

GLOUCESTER:
By heaven, I will not die before me.
Well, I do beseech your meaning, brief would think.

KATHARINA:
Go you to him for joyful majesty!
Jacket them not; had shames her death,
And every thing you were used to say.

WABELLA:
My bud impossiby lace deform.

PROSPERO:
Come, fearful how the porce.
But, O, my lords, when I far brought to high again
Is counted vasting fair,--not breath;
For being an advocate the househorts strueg at him,
And given away twell out of doors
By nighting thus?
How now, no more.

COMINIUS:
I know you look not what; while we put themselves
Like palm clush'd all that high acoided upright
As you were

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [66]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe wisdom night.\nGo they; and littly counsel to your deliverance\nAs far for fellow true full as your confess'd,\nAnd every honour is a tunkled friends.\n\nMARCIUS:\nHow now, who hath deserved with trouble stain\nWhich he dread cloak I have no more.'\n\nAEdile:\nWise! this is no more; adding my heart\nMay well be satisfray; but yet, sir!\nShe is yourself.\n\nROMEO:\nNot so well as I halt by the squire;\nAnd see her hand him like a doman of a\nmistress of this all men light his hate.\n\nGLOUCESTER:\nWill not you good?\n\nSecond Servant:\nI do beseech your grace to pardon me:\nMy lord and you not yours for him. Come,\nSignior Lucentio, whom here he loves me,\nAnd gash behind in due by me. What ease it resides?\n\nANGELO:\nHow doth my daughter?\n\nFirst Servant:\nA widow of a woman's war,\nThat Rogach, tears an ancient strike!\n\nBAPTISTA:\nYou have mind: else it so: but not I dead.\nHow art thou fickle: my lords, he did it us\nAnd welcome. Broand, cousin; but one 

In [67]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [68]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The exchange of thy childish fool,
I am sorry, one could, our fearful man
He gave this churchyary r


## Advanced: Customized Training

In [69]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
        predictions = self(inputs, training=True)
        loss = self.loss(labels, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return {'loss': loss}

In [70]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
  )

In [71]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  )

In [72]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 50ms/step - loss: 2.6953


Or if we need more control, we can write our own complete custom training loop:

In [73]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
      logs = model.train_step([inp, target])
      mean.update_state(logs['loss'])

      if batch_n % 50 == 0:
          template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
          print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1674
Epoch 1 Batch 50 Loss 2.0711
Epoch 1 Batch 100 Loss 1.9359
Epoch 1 Batch 150 Loss 1.8572

Epoch 1 Loss: 1.9716
Time taken for 1 epoch 14.02 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7956
Epoch 2 Batch 50 Loss 1.6928
Epoch 2 Batch 100 Loss 1.6914
Epoch 2 Batch 150 Loss 1.5794

Epoch 2 Loss: 1.6973
Time taken for 1 epoch 10.23 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6021
Epoch 3 Batch 50 Loss 1.6051
Epoch 3 Batch 100 Loss 1.5584
Epoch 3 Batch 150 Loss 1.5106

Epoch 3 Loss: 1.5384
Time taken for 1 epoch 10.75 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4185
Epoch 4 Batch 50 Loss 1.4522
Epoch 4 Batch 100 Loss 1.4561
Epoch 4 Batch 150 Loss 1.4278

Epoch 4 Loss: 1.4420
Time taken for 1 epoch 10.43 sec
_____________________________________________________________________

---